# Metric Presentation and Visualization
## Necessary packages and functions call

- DDPM-TS: Interpretable Diffusion for Time Series Generation
- Metrics: 
    - discriminative_metrics
    - predictive_metrics
    - visualization

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

## Data Loading

Load original dataset and preprocess the loaded data.

In [6]:
iterations = 5
ori_data = np.load('../OUTPUT/PSM_T1_finetuning/samples/PSM_T1_finetuning_ground_truth_24_train.npy')
print("ori_data_shape:",ori_data.shape)
# ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')
fake_data = np.load('../OUTPUT/PSM_T1_finetuning/ddpm_fake_PSM_T1_finetuning.npy')
print("fale_data_shape:",fake_data.shape)

ori_data_shape: (457, 24, 25)
fale_data_shape: (500, 24, 25)


## Evaluate the generated data

### 1. Discriminative score

To evaluate the classification accuracy between original and synthetic data using post-hoc RNN network. The output is | classification accuracy - 0.5 |.

- metric_iteration: the number of iterations for metric computation.

In [7]:
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:ori_data.shape[0]])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('sine:')
display_scores(discriminative_score)
print()

training: 100%|██████████| 2000/2000 [01:14<00:00, 27.02it/s]


Iter 0:  0.5 , 1.0 , 1.0 



training: 100%|██████████| 2000/2000 [01:12<00:00, 27.59it/s]


Iter 1:  0.5 , 1.0 , 1.0 



training: 100%|██████████| 2000/2000 [01:05<00:00, 30.46it/s]


Iter 2:  0.5 , 1.0 , 1.0 



training: 100%|██████████| 2000/2000 [01:01<00:00, 32.55it/s]


Iter 3:  0.5 , 1.0 , 1.0 



training: 100%|██████████| 2000/2000 [01:01<00:00, 32.34it/s]


Iter 4:  0.5 , 1.0 , 1.0 

sine:
Final Score:  0.5 ± 0.0



## Evaluate the generated data

### 2. Predictive score

To evaluate the prediction performance on train on synthetic, test on real setting. More specifically, we use Post-hoc RNN architecture to predict one-step ahead and report the performance in terms of MAE. 

The model learns to predict the last dimension with one more step.

In [8]:
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:ori_data.shape[0]])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

training: 100%|██████████| 5000/5000 [01:41<00:00, 49.10it/s]


0  epoch:  0.17576660345469292 



training: 100%|██████████| 5000/5000 [01:41<00:00, 49.26it/s]


1  epoch:  0.17453460232503248 



training: 100%|██████████| 5000/5000 [01:41<00:00, 49.17it/s]


2  epoch:  0.17516775776848478 



training: 100%|██████████| 5000/5000 [01:41<00:00, 49.37it/s]


3  epoch:  0.19123925493385396 



training: 100%|██████████| 5000/5000 [01:40<00:00, 49.79it/s]


4  epoch:  0.17262156553956234 

sine:
Final Score:  0.1778659568043253 ± 0.009397645913690406

